In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['TF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('tissue factor', 2947),
 ('transferrin', 150),
 ('tail flick', 67),
 ('theaflavin', 29),
 ('thomsen friedenreich', 27),
 ('total flavonoids', 27),
 ('transmitted founder', 26),
 ('tall fescue', 22),
 ('tolylfluanid', 18),
 ('time frequency', 14),
 ('trachomatous inflammation follicular', 13),
 ('tibiofemoral', 10),
 ('transfemoral', 9),
 ('telephone filtered', 8),
 ('typhoid fever', 8),
 ('terminal filament', 7),
 ('tension force', 7),
 ('treatment failure', 6),
 ('temporal frequency', 6),
 ('traditional foods', 5),
 ('trigger finger', 5),
 ('total fat', 5),
 ('trans fat', 5),
 ('toxicological footprint', 4),
 ('transverse foramen', 4),
 ('tube feeding', 4),
 ('triple fusion', 4),
 ('follicular trachoma', 4),
 ('tail flick reflex', 4),
 ('tear film', 3),
 ('tear fluid', 3),
 ('tf', 3),
 ('transfer function', 3),
 ('tryptophan fluorescence', 3),
 ('through focus', 3),
 ('thin filament', 3),
 ('thigh fat', 3),
 ('tongue force', 3),
 ('terfenadine', 3),
 ('transformation', 3),
 ('transf

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'HGNC:3541': 'F3',
 'HGNC:11740': 'TF',
 'CHEBI:CHEBI:136609': 'Theaflavin',
 'MESH:D005419': 'Flavonoids',
 'CHEBI:CHEBI:75182': 'Toylfluanid',
 'HP:HP:0040279': 'Frequency',
 'MESH:D014435': 'Typhoid Fever',
 'MESH:D017211': 'Treatment Failure',
 'MESH:D005502': 'Food',
 'HP:HP:0012276': 'Digital flexor tenosynovitis',
 'HGNC:1663': 'CD36',
 'MESH:D014141': 'Trachoma',
 'MESH:D012018': 'Reflex',
 'MESH:D005453': 'Fluorescence',
 'CHEBI:CHEBI:9453': 'Terfenadine'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-02 23:47:17] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'follicular trachoma': 'MESH:D014141',
  'tail flick': 'ungrounded',
  'tail flick reflex': 'ungrounded',
  'tall fescue': 'ungrounded',
  'tear film': 'ungrounded',
  'tear fluid': 'ungrounded',
  'telephone filtered': 'ungrounded',
  'temporal frequency': 'ungrounded',
  'tension force': 'ungrounded',
  'terfenadine': 'CHEBI:CHEBI:9453',
  'terminal filament': 'ungrounded',
  'tf': 'ungrounded',
  'theaflavin': 'CHEBI:CHEBI:136609',
  'thigh fat': 'ungrounded',
  'thin filament': 'ungrounded',
  'thomsen friedenreich': 'MESH:C016467',
  'through focus': 'ungrounded',
  'tibiofemoral': 'ungrounded',
  'time frequency': 'ungrounded',
  'tissue factor': 'HGNC:3541',
  'to failure': 'ungrounded',
  'tolylfluanid': 'CHEBI:CHEBI:75182',
  'tongue force': 'ungrounded',
  'total fat': 'ungrounded',
  'total flavonoids': 'ungrounded',
  'toxicological footprint': 'ungrounded',
  'trachomatous inflammation follicular': 'ungrounded',
  'traditional foods': 'MESH:D005502',
  'trans fat': 'ungr

In [11]:
grounding_map, names, pos_labels = [{'follicular trachoma': 'MESH:D014141',
  'tail flick': 'ungrounded',
  'tail flick reflex': 'ungrounded',
  'tall fescue': 'ungrounded',
  'tear film': 'ungrounded',
  'tear fluid': 'ungrounded',
  'telephone filtered': 'ungrounded',
  'temporal frequency': 'ungrounded',
  'tension force': 'ungrounded',
  'terfenadine': 'CHEBI:CHEBI:9453',
  'terminal filament': 'ungrounded',
  'tf': 'ungrounded',
  'theaflavin': 'CHEBI:CHEBI:136609',
  'thigh fat': 'ungrounded',
  'thin filament': 'ungrounded',
  'thomsen friedenreich': 'MESH:C016467',
  'through focus': 'ungrounded',
  'tibiofemoral': 'ungrounded',
  'time frequency': 'ungrounded',
  'tissue factor': 'HGNC:3541',
  'to failure': 'ungrounded',
  'tolylfluanid': 'CHEBI:CHEBI:75182',
  'tongue force': 'ungrounded',
  'total fat': 'ungrounded',
  'total flavonoids': 'ungrounded',
  'toxicological footprint': 'ungrounded',
  'trachomatous inflammation follicular': 'ungrounded',
  'traditional foods': 'MESH:D005502',
  'trans fat': 'ungrounded',
  'transfemoral': 'ungrounded',
  'transfer function': 'ungrounded',
  'transferrin': 'HGNC:11740',
  'transformation': 'ungrounded',
  'transframe': 'ungrounded',
  'transmitted founder': 'ungrounded',
  'transverse foramen': 'ungrounded',
  'treatment failure': 'MESH:D017211',
  'trigger finger': 'HP:HP:0012276',
  'triple fusion': 'ungrounded',
  'tryptophan fluorescence': 'ungrounded',
  'tube feeding': 'ungrounded',
  'typhoid fever': 'MESH:D014435'},
 {'MESH:D014141': 'Trachoma',
  'CHEBI:CHEBI:9453': 'Terfenadine',
  'CHEBI:CHEBI:136609': 'Theaflavin',
  'MESH:C016467': 'Thomsen-Friedenreich antigen',
  'HGNC:3541': 'F3',
  'CHEBI:CHEBI:75182': 'Toylfluanid',
  'MESH:D005502': 'Food',
  'HGNC:11740': 'TF',
  'MESH:D017211': 'Treatment Failure',
  'HP:HP:0012276': 'Digital flexor tenosynovitis',
  'MESH:D014435': 'Typhoid Fever'},
 ['CHEBI:CHEBI:136609',
  'CHEBI:CHEBI:75182',
  'HGNC:11740',
  'HGNC:3541',
  'MESH:C016467']]

In [12]:
excluded_longforms = ['tf']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[]

In [20]:
intersection2

[]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-03 00:37:42] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-03 00:38:30] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9746253202407752 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'HGNC:3541': 2351,
  'HGNC:11740': 114,
  'ungrounded': 184,
  'MESH:D014141': 4,
  'CHEBI:CHEBI:9453': 3,
  'MESH:D014435': 6,
  'MESH:C016467': 19,
  'CHEBI:CHEBI:136609': 23,
  'CHEBI:CHEBI:75182': 10,
  'MESH:D017211': 4,
  'HP:HP:0012276': 4,
  'MESH:D005502': 4},
 'f1': {'mean': 0.974625, 'std': 0.006322},
 'precision': {'mean': 0.968984, 'std': 0.009347},
 'recall': {'mean': 0.982915, 'std': 0.005709},
 'CHEBI:CHEBI:9453': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D014435': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'MESH:C016467': {'f1': {'mean': 0.674286, 'std': 0.372351},
  'pr': {'mean': 0.633333, 'std': 0.371184},
  'rc': {'mean': 0.733333, 'std': 0.38873}},
 'MESH:D005502': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'ungrounded

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for TF

Produces the disambiguations:
	Digital flexor tenosynovitis	HP:HP:0012276
	F3*	HGNC:3541
	Food	MESH:D005502
	TF*	HGNC:11740
	Terfenadine	CHEBI:CHEBI:9453
	Theaflavin*	CHEBI:CHEBI:136609
	Thomsen-Friedenreich antigen*	MESH:C016467
	Toylfluanid*	CHEBI:CHEBI:75182
	Trachoma	MESH:D014141
	Treatment Failure	MESH:D017211
	Typhoid Fever	MESH:D014435

Class level metrics:
--------------------
Grounding                   	Count	F1     
                          F3*	2351	 0.9847
                  Ungrounded	 184	0.87679
                          TF*	 114	0.87217
                  Theaflavin*	  23	0.76905
Thomsen-Friedenreich antigen*	  19	0.67429
                 Toylfluanid*	  10	0.86667
               Typhoid Fever	   6	    0.6
                    Trachoma	   4	    0.0
           Treatment Failure	   4	    0.0
Digital flexor tenosynovitis	   4	    0.0
                        Food	   4	    0.4
                 Terfenadine	   3	    0.0

Weighted Metrics:
------------

In [26]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1